In [1]:
import torch
import numpy as np

from transformers.models.wav2vec2.modeling_wav2vec2 import _compute_mask_indices, _sample_negative_indices
from datasets import load_dataset
from lightning.pytorch import seed_everything
from transformers import Wav2Vec2Config
import seisbench.data as sbd

# from transformers import AutoFeatureExtractor, Wav2Vec2ForPreTraining
from transformers import Wav2Vec2ForPreTraining as RefWav2Vec2ForPreTraining
from seisLM.model.wav2vec2 import Wav2Vec2ForPreTraining

data = sbd.STEAD()

config = Wav2Vec2Config.from_pretrained("facebook/wav2vec2-base")
seed_everything(0)
ref_model = RefWav2Vec2ForPreTraining(config)
model = Wav2Vec2ForPreTraining(config)
model.load_state_dict(ref_model.state_dict())


waveforms = data.get_waveforms(1265656)
input_values = torch.Tensor(waveforms[0]).unsqueeze(0)

# compute masked indices
batch_size, raw_sequence_length = input_values.shape
sequence_length = model._get_feat_extract_output_lengths(raw_sequence_length).item()

seed_everything(0)
mask_time_indices = _compute_mask_indices(
    shape=(batch_size, sequence_length), mask_prob=0.2, mask_length=2
)
sampled_negative_indices = _sample_negative_indices(
    features_shape=(batch_size, sequence_length),
    num_negatives=model.config.num_negatives,
    mask_time_indices=mask_time_indices,
)
mask_time_indices = torch.tensor(data=mask_time_indices, device=input_values.device, dtype=torch.long)
sampled_negative_indices = torch.tensor(
    data=sampled_negative_indices, device=input_values.device, dtype=torch.long
)

with torch.no_grad():
    outputs = model(input_values, mask_time_indices=mask_time_indices,
                    sampled_negative_indices=sampled_negative_indices)


# print(f'projected_states mean {outputs.projected_states.mean()}')
# print(f'projected_quantized_states mean {outputs.projected_quantized_states.mean()}')
# print(f'codevector_perplexity mean {outputs.codevector_perplexity.mean()}')

np.testing.assert_almost_equal(
  outputs.projected_states.mean(), -0.02380690723657608
)
np.testing.assert_almost_equal(
  outputs.projected_quantized_states.mean(), 0.017036495730280876
)
np.testing.assert_almost_equal(
  outputs.codevector_perplexity.mean(), 11.194181442260742
)



2024-05-24 18:31:38,689 | seisbench | WARNING | Component order not specified, defaulting to 'ZNE'.
/home/liu0003/miniconda3/envs/seisbench/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/liu0003/miniconda3/envs/seisbench/lib/python3.9/site-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Seed set to 0
Seed set to 0


In [2]:
config.num_negatives

100

In [3]:
# sum_mean = 0
# for name, param in list(model.named_parameters()):
#   print(f'{name}, param mean: {param.mean()}')
#   sum_mean += param.mean()


In [4]:
# sum_mean = 0
# for name, param in list(model.named_parameters()):
#   print(f'{name}, param mean: {param.mean()}')
#   sum_mean += param.mean()

# print(sum_mean)


In [5]:
# sum_mean